# Attempt at Cape Cod Model (from SWI2 examples)

In [ ]:
import pathlib as pl
import shutil
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import flopy

# path to mf6 shared library
# executables based on mf6 feat-swi branch at: 
#   https://github.com/langevin-usgs/modflow6/tree/feat-swi-correction

# Put the name of the mf6 executable into mf6exe.txt,
# which is not under version control.
with open(pl.Path("./mf6exe.txt"), "r") as f:
    mf6exe = f.readline().strip()
print(f"using executable: {mf6exe}")

# data path contains the mf2005 Cape Cod SWI2 model
data_pth = pl.Path("../data/swi2ex07")
sim_ws = pl.Path("./temp/case6")

In [ ]:
# copy the model into ./temp/case6/mf2005
src = data_pth
mf2005_path = sim_ws / "mf2005"
shutil.copytree(src, mf2005_path, dirs_exist_ok=True)
print(mf2005_path.resolve())

In [ ]:
# load the mf2005 swi2 package to get starting zeta
f = "swi2ex7.nam"
m = flopy.modflow.Modflow.load(f, load_only=["SWI2"], model_ws=mf2005_path)
zetastrt = m.swi2.zeta[0].array

In [ ]:
# run the mf5to6 converter
cmd_list = ["mf5to6", "swi2ex7.nam", "mf6"]
subprocess.run(
    cmd_list,
    cwd=str(mf2005_path.resolve()),
)

In [ ]:
# copy mf6 files into mf6 folder
mf6_files = [x.parts[-1] for x in mf2005_path.glob("*")]
mf6_files = [x for x in mf6_files if x.startswith("mf6")] + ["mfsim.nam"]
mf6_files

mf6_path = sim_ws / "mf6"
mf6_path.mkdir(parents=True, exist_ok=True)

for f in mf6_files:
    src = mf2005_path / f
    dst = mf6_path / f
    print (f"copying {src} => {dst}")
    shutil.copyfile(src, dst)

In [ ]:
# run the 2005 version of the model? (this takes a while...)
if False:
    cmd_list = ["mf2005", "swi2ex7.nam"]
    return_code = subprocess.call(cmd_list, shell=True, cwd=mf2005_path)
    print(return_code)

In [ ]:
# overwrite ims
txt = """BEGIN options
  PRINT_OPTION ALL
END options

BEGIN nonlinear
  OUTER_MAXIMUM  500
  OUTER_DVCLOSE 0.01
END nonlinear

BEGIN linear
  INNER_DVCLOSE 0.01
  LINEAR_ACCELERATION bicgstab
END linear
"""
fname = mf6_path / "mf6.ims"
with open(fname, "w") as f:
    f.write(txt)

# overwrite tdis
txt = """BEGIN Options
  TIME_UNITS  days
END Options

BEGIN Dimensions
  NPER  1
END Dimensions

BEGIN PERIODDATA
   18200.00000000  1       1.00000000
END PERIODDATA
"""
fname = mf6_path / "mf6.tdis"
with open(fname, "w") as f:
    f.write(txt)

# overwrite oc
txt = """BEGIN Options
  BUDGET  FILEOUT  mf6.bud
  HEAD  FILEOUT  mf6.hds
END Options

BEGIN PERIOD  1
  SAVE  budget all
  SAVE  head  all
  PRINT  budget all
END PERIOD  1
"""
fname = mf6_path / "mf6.oc"
with open(fname, "w") as f:
    f.write(txt)

In [ ]:
# load the simulation
sim = flopy.mf6.MFSimulation.load(sim_ws=mf6_path, exe_name=mf6exe)
gwf = sim.gwf[0]
gwf.newtonoptions = "NEWTON"
botm = gwf.dis.botm.array

npf = gwf.npf
npf.perched = False
npf.cvoptions = None

zeta_file = gwf.name + '.zta'
swi = flopy.mf6.ModflowGwfswi(gwf, zeta_filerecord=zeta_file, zetastrt=botm)

sim.write_simulation()
sim.run_simulation()

In [ ]:
head_all = gwf.output.head().get_alldata()
head = head_all[-1]
#head[head < 1e30] /= 3.2808

pmv = flopy.plot.PlotMapView(model=gwf)
cb = pmv.plot_array(head)
#pmv.contour_array(head, levels=[5, 10, 15], colors="k")
plt.colorbar(cb, shrink=0.5)

jcol = 550
x = gwf.modelgrid.xcellcenters[-1, jcol], gwf.modelgrid.xcellcenters[0, jcol]
y = gwf.modelgrid.ycellcenters[-1, jcol], gwf.modelgrid.ycellcenters[0, jcol]
pmv.ax.plot(x, y, 'r-')
x, y

In [ ]:
fpth = mf6_path / f"{gwf.name}.zta"
zeta_all = flopy.utils.HeadFile(fpth, text="zeta").get_alldata()
zeta = zeta_all[-1].flatten()

pmv = flopy.plot.PlotMapView(model=gwf)
cb = pmv.plot_array(zeta)
plt.colorbar(cb, shrink=0.5)


In [ ]:
# Guestimate the column number used in the SWI2 report for A-A'
jcol = 550
istart = np.where(head[0, :, jcol] < 1.e30)[0][0]
istop =  np.where(head[0, :, jcol] < 1.e30)[0][-1]

x = gwf.modelgrid.ycellcenters[:, jcol]
x = x[0] - x
x = x[istart:istop]

fig = plt.figure(figsize=(10, 4))
ax = fig.add_subplot(1, 1, 1)

# top
t = gwf.dis.top.array[:, jcol]
t = t[istart:istop]
ax.plot(x, t)

# bot
b = gwf.dis.botm.array[0, :, jcol]
b = b[istart:istop]

# head
h = head[0, :, jcol]
h[h==1.e30] = np.nan
h = h[istart:istop]

# zeta
z = zeta_all[-1][0, :, jcol]
z[z==1.e30] = np.nan
z = z[istart:istop]

# fill
ax.fill_between(x, -350, b, color="gray")
ax.fill_between(x, b, z, color="red")
ax.fill_between(x, z, h, color="cyan")
